<a href="https://colab.research.google.com/github/Divyajoyt/Pyspark/blob/main/crimes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
!tar xf spark-2.3.1-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"

!ls

import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate() 
spark

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [67.0 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:15 http://sec

In [3]:
mydata = spark.read.format("csv").option("header","true").load("crimes.csv")
mydata.show()

+--------+---------+--------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+-------------+-------+-----------+-----------+----+---------+---------+----------+-------------+
|      ID|     Date|               Block|IUCR|        PrimaryType|         Description| LocationDescription|Arrest|Domestic|Beat|District|Ward|CommunityArea|FBICode|XCoordinate|YCoordinate|Year|UpdatedOn| Latitude| Longitude|     Location|
+--------+---------+--------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+-------------+-------+-----------+-----------+----+---------+---------+----------+-------------+
|11094370|18-Jun-21|072XX N CALIFORNI...|1122| DECEPTIVE PRACTICE|   COUNTERFEIT CHECK|   CURRENCY EXCHANGE|  TRUE|   FALSE|2411|      24|  50|            2|     10|    1156443|    1947707|2021|18-Jun-21|42.012293|-87.699714|   CALIFORNIA|
|11118031|23-Mar-21|   055XX W GRAND AVE

In [6]:
from pyspark.sql.types import *
myschema = StructType([
                       StructField('ID', IntegerType()),
                       StructField('Date', StringType()),
                       StructField('Block', StringType()),
                       StructField('IUCR', IntegerType()),
                       StructField('Primary Type', StringType()),
                       StructField('Description', StringType()),
                       StructField('LocationDescription', StringType()),
                       StructField('Arrest', StringType()),
                       StructField('Domestic', StringType()),
                       StructField('Beat', IntegerType()),
                       StructField('District', IntegerType()),
                       StructField('Ward', IntegerType()),
                       StructField('Community', IntegerType()),
                       StructField('FBICode', IntegerType()),
                       StructField('XCoordinate', FloatType()),
                       StructField('YCoordinate', FloatType()),
                       StructField('Year', IntegerType()),
                       StructField('UpdatedOn', StringType()),
                       StructField('Latitude', FloatType()),
                       StructField('Longitude', FloatType()),
                       StructField('Location', StringType()),
])


In [8]:
df = spark.read.csv("crimes.csv", header=True, schema=myschema)
df.show()

+--------+---------+--------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+---------+-------+-----------+-----------+----+---------+---------+----------+-------------+
|      ID|     Date|               Block|IUCR|       Primary Type|         Description| LocationDescription|Arrest|Domestic|Beat|District|Ward|Community|FBICode|XCoordinate|YCoordinate|Year|UpdatedOn| Latitude| Longitude|     Location|
+--------+---------+--------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+---------+-------+-----------+-----------+----+---------+---------+----------+-------------+
|11094370|18-Jun-21|072XX N CALIFORNI...|1122| DECEPTIVE PRACTICE|   COUNTERFEIT CHECK|   CURRENCY EXCHANGE|  TRUE|   FALSE|2411|      24|  50|        2|     10|  1156443.0|  1947707.0|2021|18-Jun-21|42.012295|-87.699715|   CALIFORNIA|
|11118031|23-Mar-21|   055XX W GRAND AVE|1345|    CRIMIN

In [13]:
df.registerTempTable("crimes")

query1 = spark.sql("select * from crimes where LocationDescription='RESIDENCE' order by IUCR desc")
query1.show()

+--------+---------+-------------------+----+-------------------+--------------------+-------------------+------+--------+----+--------+----+---------+-------+-----------+-----------+----+---------+---------+---------+----------+
|      ID|     Date|              Block|IUCR|       Primary Type|         Description|LocationDescription|Arrest|Domestic|Beat|District|Ward|Community|FBICode|XCoordinate|YCoordinate|Year|UpdatedOn| Latitude|Longitude|  Location|
+--------+---------+-------------------+----+-------------------+--------------------+-------------------+------+--------+----+--------+----+---------+-------+-----------+-----------+----+---------+---------+---------+----------+
|11134189|11-Feb-21| 043XX S TALMAN AVE|4651|      OTHER OFFENSE|SEX OFFENDER: FAI...|          RESIDENCE|  TRUE|   FALSE| 922|       9|  12|       58|     26|  1159425.0|  1875711.0|2021|11-Feb-21| 41.81467|-87.69073|   BELMONT|
|45789000|09-Apr-21|022XX W CHICAGO AVE| 768|          VANDALISM|           VAND

In [21]:
df.write.json('CrimesOutput.json', compression='gzip', mode='overwrite')